In [ ]:
%load_ext autoreload
%autoreload 2
import time
from datetime import datetime, date, time, timedelta
import asyncio
import nest_asyncio
import importlib
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
from tastytrade import Account, Session, DXLinkStreamer
from tastytrade.dxfeed import Quote, Candle
from tastytrade.instruments import *
from tastytrade.market_data import *
from config import *
from alphas import *
from itertools import chain

nest_asyncio.apply()

In [79]:
import tastytrade.market_data
importlib.reload(tastytrade.market_data)

<module 'tastytrade.market_data' from 'C:\\Users\\warre\\PycharmProjects\\tastytrade-bot\\.venv\\Lib\\site-packages\\tastytrade\\market_data.py'>

In [ ]:
session=None
account=None

async def setup():
    global session, account
    # setup
    config = Config(test=False)
    session = Session(config.username, config.password, is_test=config.test)
    account = await Account.a_get(session, config.account_number)

    # account info
    balance = account.get_balances(session)
    positions = account.get_positions(session)
    history = account.get_history(session)

asyncio.run(setup())

In [ ]:
streamer = DXLinkStreamer(session) #async streamer

SPXoptionchain = get_option_chain(session, "SPX")
VIXoptionchain = get_option_chain(session, "VIX")

SPXexpiries = list(SPXoptionchain.keys())
VIXexpiries = list(VIXoptionchain.keys())
#print(SPXexpiries)

In [ ]:
VXfutures = Future.get(session, symbols = None, product_codes=["VX"])
VXfm = VXfutures[-1]
#VXbm = VXfutures[-2]

In [ ]:
VXfm

In [ ]:
futuresreq = [VXfm.symbol]
VXfmdata = get_market_data_by_type(session,futures=futuresreq)
print(VXfmdata)

In [75]:
SPXoptionchain[date(2025, 12, 31)]

[Option(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'> symbol='SPXW  251231C01200000' active=True strike_price=Decimal('1200.0') root_symbol='SPXW' underlying_symbol='SPX' expiration_date=datetime.date(2025, 12, 31) exercise_style='European' shares_per_contract=100 option_type=<OptionType.CALL: 'C'> option_chain_type='Standard' expiration_type='Quarterly' settlement_type='PM' stops_trading_at=datetime.datetime(2025, 12, 31, 21, 0, tzinfo=TzInfo(UTC)) market_time_instrument_collection='Cash Settled Equity Option' days_to_expiration=246 expires_at=datetime.datetime(2025, 12, 31, 21, 0, tzinfo=TzInfo(UTC)) streamer_symbol='.SPXW251231C1200'),
 Option(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'> symbol='SPXW  251231P01200000' active=True strike_price=Decimal('1200.0') root_symbol='SPXW' underlying_symbol='SPX' expiration_date=datetime.date(2025, 12, 31) exercise_style='European' shares_per_contract=100 option_type=<OptionType.PUT: 'P'> option_chain_type=

In [80]:
get_market_data_by_type(session, options = ['SPXW  251231P05020000'])

[MarketData(symbol='SPXW  251231P05020000' instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'> updated_at=datetime.datetime(2025, 4, 30, 2, 49, 2, 885000, tzinfo=TzInfo(UTC)) bid_size=Decimal('18.0') ask_size=Decimal('18.0') mark=Decimal('178.15') close_price_type=<ClosePriceType.REGULAR: 'Regular'> prev_close_price_type=<ClosePriceType.REGULAR: 'Regular'> summary_date=datetime.date(2025, 4, 30) prev_close_date=datetime.date(1970, 1, 1) halt_start_time=-1 halt_end_time=-1 ask=Decimal('180.1') bid=Decimal('176.2') mid=Decimal('178.15'))]

In [ ]:
a=SPXoptionchain[date(2025,12,31)][0].option_type
print(a,type(a))

In [ ]:
VIXoptionchain[date(2025,5,21)]

In [ ]:
#rfr needs to be taken from yfinance tt does not have
rfr = 5

In [ ]:
indexreq = ["SPX"]
SPXdata = get_market_data_by_type(session,indices = indexreq)
print(type(SPXdata[0].last))

In [72]:
def calc_forward_iv(chain, dte, rfr, spot):  # need specifically the chain of the day
    T = dte / 365
    r = rfr / 100
    forward = spot * np.exp(r * T)
    chain = [opt for opt in chain if opt.settlement_type == "PM"]
    chain = convertchain(chain)

    K0 = chain.loc[abs(chain["strike"] - forward).idxmin(), "strike"]

    otmcalls = chain[chain["strike"] >= K0]
    otmputs = chain[chain["strike"] <= K0]
    call_bound = otmcalls[
        ((otmcalls["cbid"] == 0) & (otmcalls.shift(-1)["cbid"] == 0))
        | ((otmcalls["cask"] == 0) & (otmcalls.shift(-1)["cask"] == 0))
        | (otmcalls["cbid"].isna())
        | (otmcalls["cask"].isna())
    ].index.min()
    put_bound = otmputs[
        ((otmputs["pbid"] == 0) & (otmputs.shift(1)["pbid"] == 0))
        | ((otmputs["pask"] == 0) & (otmputs.shift(1)["pask"] == 0))
        | (otmputs["pbid"].isna())
        | (otmputs["pask"].isna())
    ].index.max()

    if pd.notna(call_bound):
        chain = chain.loc[: call_bound - 1]  # right end is inclusive in pd loc
    if pd.notna(put_bound):
        chain = chain.loc[put_bound + 1 :]
    if otmcalls.empty or otmputs.empty:
        raise ValueError(
            f"no OTM options check code for {dte} dte"
        )

    chain = chain[
        ~(
            (
                (chain["strike"] >= K0)
                & ((chain["cbid"] == 0) | (chain["cask"] == 0))
            )
            | (
                (chain["strike"] <= K0)
                & ((chain["pbid"] == 0) | (chain["pask"] == 0))
            )
        )
    ]  # ignoring rows that have either 0 bid or ask

    chain["dK"] = (chain["strike"].shift(-1) - chain["strike"].shift(1)) / 2
    chain.loc[chain.index[0], "dK"] = (
        chain.loc[chain.index[1], "strike"] - chain.loc[chain.index[0], "strike"]
    )
    chain.loc[chain.index[-1], "dK"] = (
        chain.loc[chain.index[-1], "strike"]
        - chain.loc[chain.index[-2], "strike"]
    )
    otmcalls = chain[chain["strike"] >= K0]
    otmputs = chain[chain["strike"] <= K0]

    call_vals = (
        (
            otmcalls[otmcalls["strike"] != K0]["cbid"]
            + otmcalls[otmcalls["strike"] != K0]["cask"]
        )
        / 2
        * (
            otmcalls[otmcalls["strike"] != K0]["dK"]
            / otmcalls[otmcalls["strike"] != K0]["strike"] ** 2
        )
    )
    put_vals = (
        (
            otmputs[otmputs["strike"] != K0]["pbid"]
            + otmputs[otmputs["strike"] != K0]["pask"]
        )
        / 2
        * (
            otmputs[otmputs["strike"] != K0]["dK"]
            / otmputs[otmputs["strike"] != K0]["strike"] ** 2
        )
    )
    forwardiv = (
        2
        / T
        * np.exp(r * T)
        * (
            np.sum(call_vals)
            + interpolate(otmcalls, "call", "fiv")
            + np.sum(put_vals)
            + interpolate(otmputs, "put", "fiv")
            + (otmcalls.iloc[0]["cbid"] + otmcalls.iloc[0]["cask"])
            / 4
            * (otmcalls.iloc[0]["dK"] / otmcalls.iloc[0]["strike"] ** 2)
            + (otmputs.iloc[-1]["pbid"] + otmputs.iloc[-1]["pask"])
            / 4
            * (otmputs.iloc[-1]["dK"] / otmputs.iloc[-1]["strike"] ** 2)
        )
        - 1 / T * (forward / K0 - 1) ** 2
    )

    return (
        10000 * forwardiv
    )  # 100^2, since varq is measured on vix options, therefore this also needs to be scaled to actual vix


def calc_varq(chain, dte, rfr, forward):
    # in varq the chain is the vix chain and spot is spot vix
    calls_bounded, puts_bounded = False, False
    T = dte / 365
    r = rfr / 100
    chain = convertchain(chain)
    chain["dK"] = (chain["strike"].shift(-1) - chain["strike"].shift(1)) / 2
    chain.loc[chain.index[0], "dK"] = (
        chain.loc[chain.index[1], "strike"] - chain.loc[chain.index[0], "strike"]
    )
    chain.loc[chain.index[-1], "dK"] = (
        chain.loc[chain.index[-1], "strike"]
        - chain.loc[chain.index[-2], "strike"]
    )
    K0 = chain.loc[abs(chain["strike"] - forward).idxmin(), "strike"]
    otmcalls = chain[chain["strike"] >= K0]
    otmputs = chain[chain["strike"] <= K0]
    call_bound = otmcalls[
        (otmcalls["cbid"] == 0)
        | (otmcalls["cask"] == 0)
        | (otmcalls["cbid"].isna())
        | (otmcalls["cask"].isna())
    ].index.min()
    put_bound = otmputs[
        (otmputs["pbid"] == 0)
        | (otmputs["pask"] == 0)
        | (otmputs["pbid"].isna())
        | (otmputs["pask"].isna())
    ].index.max()
    if pd.notna(call_bound):
        otmcalls = otmcalls.loc[: call_bound - 1]
        calls_bounded = True
    if pd.notna(put_bound):
        otmputs = otmputs.loc[put_bound + 1 :]
        puts_bounded = True
    if calls_bounded == False:
        print("vix calls unbounded")
    if puts_bounded == False:
        print("vix puts unbounded")

    if otmcalls.empty and otmputs.empty:
        raise ValueError(
            f"no OTM options on VIX check code for {dte} dte"
        )
    elif otmputs.empty and not otmcalls.empty:
        # print(f"no OTM puts on VIX on {chain.loc[:,' [QUOTE_DATE]'].iloc[0]} exp {chain.loc[:,' [EXPIRE_DATE]'].iloc[0]}")
        call_vals = (otmcalls["cbid"] + otmcalls["cask"]) / 2 * otmcalls["dK"]
        varq = 2 * np.exp(r * T) * (np.sum(call_vals)+(interpolate(otmcalls, "call", "varq") if not calls_bounded else 0))
    elif otmcalls.empty and not otmputs.empty:
        put_vals = (otmputs["pbid"] + otmputs["pask"]) / 2 * otmputs["dK"]
        varq = 2 * np.exp(r * T) * (np.sum(put_vals)+(interpolate(otmputs, "put", "varq") if not puts_bounded else 0))
    else:
        call_vals = (
            (
                otmcalls[otmcalls["strike"] != K0]["cbid"]
                + otmcalls[otmcalls["strike"] != K0]["cask"]
            )
            / 2
            * otmcalls[otmcalls["strike"] != K0]["dK"]
        )
        put_vals = (
            (
                otmputs[otmputs["strike"] != K0]["pbid"]
                + otmputs[otmputs["strike"] != K0]["pask"]
            )
            / 2
            * otmputs[otmputs["strike"] != K0]["dK"]
        )
        varq = (
            2
            * np.exp(r * T)
            * (
                np.sum(call_vals)
                + np.sum(put_vals)
                +(interpolate(otmcalls, "call", "varq") if not calls_bounded else 0)
                +(interpolate(otmputs, "put", "varq") if not puts_bounded else 0)
                + (otmcalls.iloc[0]["cbid"] + otmcalls.iloc[0]["cask"])
                / 4
                * otmcalls.iloc[0]["dK"]
                + (otmputs.iloc[-1]["pbid"] + otmputs.iloc[-1]["pask"])
                / 4
                * otmputs.iloc[-1]["dK"]
            )
        )
    return varq

def interpolate(chain, type, component):
    # sometimes, the chain gets "cut off" because there are not enough strikes. this results in an incorrect calculation of the implied vix future value
    # there are no easy workarounds. here are the few things we can do:
    # 1. ignore expiry if we cannot find call_bound or put_bound. however sometimes we cannot simply skip an expiry day (sometimes we do not have the luxury of finding another expiry day that works for our vx expiry)
    # 2. choosing only monthlies, but this is considerably less accurate, and in older years sometimes even monthlies do not have reliable strikes
    # 3. interpolating otm option prices, but this introduces errors due to assumptions being used in the distribution of prices. however, this is the most reasonable approach as of now
    # this interpolation function will attempt to estimate the size of the tail that is "cut off" from the option chain, and return 0 if the tail is not cut off (ie bound exists)

    # print(f'interpolating {chain.loc[:," [QUOTE_DATE]"].iloc[0]} exp {chain.loc[:," [EXPIRE_DATE]"].iloc[0]}')
    if len(chain) == 0:
        raise ValueError(
            f"interpolating null chain {chain.loc[:," [QUOTE_DATE]"].iloc[0]} exp {chain.loc[:," [EXPIRE_DATE]"].iloc[0]}"
        )
    elif len(chain) == 1:
        print(
            f"unable to interpolate chain with 1 data point {chain.loc[:,' [QUOTE_DATE]'].iloc[0]} exp {chain.loc[:,' [EXPIRE_DATE]'].iloc[0]}"
        )
        return 0
    else:
        if type == "call":
            cutoff = chain.iloc[-1]
            cutoffminus1 = chain.iloc[-2]
            c1 = cutoff["strike"]
            f1 = (cutoff["cbid"] + cutoff["cask"]) / 2
            x = cutoff["strike"] - cutoffminus1["strike"]
            y = (cutoff["cbid"] + cutoff["cask"]) / 2 - (
                cutoffminus1["cbid"] + cutoffminus1["cask"]
            ) / 2
            if round(y, 3) >= 0:
                f1 = (cutoffminus1["cbid"] + cutoffminus1["cask"]) / 2
                y = -f1
            if component == "fiv":
                tail = max(-(f1**2 *x )/(2 * c1**2 * y),0)
            elif component == "varq":
                tail = max(f1 - c1*f1 - (f1**2 * x)/(2 * y), 0)
            else:
                raise ValueError(f"invalid component {component}")
        elif type == "put":
            cutoff = chain.iloc[0]
            cutoffplus1 = chain.iloc[1]
            c1 = cutoff["strike"]
            f1 = (cutoff["pbid"] + cutoff["pask"]) / 2
            x = cutoff["strike"] - cutoffplus1["strike"]
            y = (cutoff["pbid"] + cutoff["pask"]) / 2 - (
                cutoffplus1["pbid"] + cutoffplus1["pask"]
            ) / 2
            if round(y, 3) >= 0:
                f1 = (cutoffplus1["pbid"] + cutoffplus1["pask"]) / 2
                y = f1
            if component == "fiv":
                tail = max((f1**2 *x )/(2 * c1**2 * y),0)
            elif component == "varq":
                tail = max(-(f1 - c1*f1 - (f1**2 * x)/(2 * y)), 0)
            else:
                raise ValueError(f"invalid component {component}")
        else:
            raise ValueError(f"invalid type {type}")
        # print(f'{dte} dte {type} adj tail {tail * 2/dte * np.exp(rfr * dte) * 10000} bound {cutoff[' [STRIKE]']} terms {(y/x) - (y*c1/x + y/2)/(c1 + x/2) - (y/(2*x)) * ((x/2 + f1 * (x/y))/(c1 + x/2))**2}, {2/dte}, {rfr} ')
        return tail

def batch_findoption(items, func, batch_size=90):
    # generator of batches
    batches = (
        items[i : i + batch_size]
        for i in range(0, len(items), batch_size)
    )
    return list(chain.from_iterable(func([option.symbol for option in batch]) for batch in batches))

def fetch_options(option_batch):
    optiondata = get_market_data_by_type(session, options = option_batch)
    return optiondata

def convertchain(chain):
    all_results = batch_findoption(chain, fetch_options, batch_size=90)

    res_map = {res.symbol: res for res in all_results}

    combined = [
        {
            "symbol":       opt.symbol,
            "strike_price": float(opt.strike_price),
            "option_type":  opt.option_type,
            "bid":          float(getattr(res_map.get(opt.symbol, None), "bid",  np.nan)),
            "ask":          float(getattr(res_map.get(opt.symbol, None), "ask",  np.nan)),
        }
        for opt in chain
    ]

    df = pd.DataFrame(combined)
    vc = df["strike_price"].value_counts()
    # find any strikes that don’t have exactly 2 entries
    wrong = vc[vc != 2]

    # assert, printing out the offending strike_price values
    assert wrong.empty, f"Strikes without exactly two entries: {wrong.index.tolist()}"

    df["type"] = df["option_type"].apply(
        lambda ot: "C" if ot == OptionType.CALL
                   else "P" if ot == OptionType.PUT
                   else (_ for _ in ()).throw(ValueError(f"Unexpected OptionType: {ot!r}"))
    )

    df_pivot = (
        df
        .pivot(index="strike_price", columns="type", values=["bid", "ask"])
        .reset_index()
    )

    df_pivot.columns = [
        f"{col_type.lower()}{val}" if isinstance(col_type, str)
        else col_type  # this picks up the strike_price index as-is
        for val, col_type in df_pivot.columns
    ]

    # 7. Rename strike_price → strike, reorder
    dfchain = (
        df_pivot
        .rename(columns={"strike_price": "strike"})
        [["cbid", "cask", "strike", "pbid", "pask"]]
    )
    return dfchain

In [ ]:
test = [opt.symbol for opt in VIXoptionchain[date(2025,5,21)]][:10]
print(test)
get_market_data_by_type(session, options = test)
#get_market_data_by_type(session, options=['SPXW  250501C05490000'])

In [70]:
convertchain(VIXoptionchain[date(2025,5,21)])

142
                  symbol  strike_price      option_type    bid    ask
0  VIX   250521C00010000          10.0  OptionType.CALL  13.45  13.60
1  VIX   250521P00010000          10.0   OptionType.PUT   0.00   0.02
2  VIX   250521C00010500          10.5  OptionType.CALL  12.95  13.10
3  VIX   250521P00010500          10.5   OptionType.PUT   0.00   0.02
4  VIX   250521C00011000          11.0  OptionType.CALL  12.45  12.60


,cbid,cask,strike,pbid,pask
0,13.45,13.60,10.0,0.00,0.02
1,12.95,13.10,10.5,0.00,0.02
2,12.45,12.60,11.0,0.00,0.02
3,11.95,12.10,11.5,0.00,0.02
4,11.45,11.60,12.0,0.00,0.02
...,...,...,...,...,...
66,0.00,0.04,160.0,136.00,136.20
67,0.00,0.04,170.0,146.00,146.15
68,0.00,0.04,180.0,155.95,156.15
69,0.00,0.03,190.0,165.90,166.10


In [74]:
vx1expiryf = VXfm.last_trade_date
vx1expiryb = vx1expiryf + timedelta(days=30)
today = date.today()
ervart1values = []
values = {
        "ervart1": pd.NA,
        "varqt1": pd.NA,
        "eqv1": pd.NA,
    }

spot = float(SPXdata[0].last)
VXstyle = "prev-close"
if VXstyle == "last":
    vx1close = float(VXfmdata[0].last)
elif VXstyle == "prev-close":
    vx1close = float(VXfmdata[0].prev_close)
else:
    raise ValueError(f"invalid style {VXstyle}")

for vx1expiry in [vx1expiryf, vx1expiryb]:
    t1 = (vx1expiry - today).days
    if vx1expiry in SPXexpiries:
        print(vx1expiry)
        t1chain = SPXoptionchain[vx1expiry]

        ervart1 = calc_forward_iv(
            t1chain, t1, rfr, spot
        )
    else:
        if (
            SPXexpiries[0] > vx1expiry
        ):  # sometimes the closest vx futures expiry is before the closest spx option expiry, this happens in earlier years
            exp1 = today  # the workaround is to let the expiry be today to "simulate" a 0dte that expired at close today
        else:
            exp1 = exp1 = max([d for d in SPXexpiries if d < vx1expiry], default=None)
        exp2 = min([d for d in SPXexpiries if d > vx1expiry], default=None)
        print(exp1, exp2)
        t1e1 = (exp1 - date).days  # front expiry of this vix future
        t1e2 = (
            exp2 - date
        ).days  # back expiry of this vix future, does NOT refer to the expiry of either the front or back timestamps in vx1expiryf or b
        if date == exp1:
            vart1e1 = 0
            # print("date = exp1 for vx1") #debug line
        else:
            t1e1chain = SPXoptionchain[exp1]
            vart1e1 = calc_forward_iv(
                t1e1chain, t1e1, rfr, spot
            )
        t1e2chain = SPXoptionchain[exp2]
        vart1e2 = calc_forward_iv(
            t1e2chain, t1e2, rfr, spot
        )
        ervart1 = ((t1e2 - t1) * t1e1 * vart1e1 + (t1 - t1e1) * t1e2 * vart1e2) / (
            (t1e2 - t1e1) * t1
        )  # https://gregorygundersen.com/blog/2023/09/10/deriving-vix/
    ervart1values.append((t1, ervart1))
ervart1 = (
    ervart1values[1][0] * ervart1values[1][1]
    - ervart1values[0][0] * ervart1values[0][1]
) / (
    ervart1values[1][0] - ervart1values[0][0]
)  # (Tb * vb - Ta * va) / (Tb - Ta)
values["ervart1"] = ervart1

assert vx1expiryf in VIXexpiries
t1vixchain = VIXoptionchain[vx1expiryf]
print(vx1expiryf)
varqt1 = calc_varq(t1vixchain, (vx1expiryf - today).days, rfr, vx1close)
values["varqt1"] = varqt1

if pd.notna(ervart1) and pd.notna(varqt1):
    values["eqv1"] = np.sqrt(ervart1 - varqt1)

print(values)
print(f'prev VX1 close {VXfmdata[0].prev_close} current price {VXfmdata[0].last}')

2025-05-21
2025-06-20
2025-05-21
{'ervart1': np.float64(624.1027578154516), 'varqt1': np.float64(60.27815373184738), 'eqv1': np.float64(23.74499113673459)}
prev VX1 close 23.4 current price 23.5


In [ ]:
VIXexpiries

In [ ]:
VXfm.last_trade_date

In [ ]:
VXfutures[0].last_trade_date